In [7]:
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi]==2.10.0

!git clone -l -s git://github.com/wjwswain/chaRL.git chaRL
%cd chaRL/games/blackjack
from env import GameEnv

import gym
import numpy as np
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import A2C

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into 'chaRL'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 61 (delta 11), reused 52 (delta 5), pack-reused 0
Receiving objects: 100% (61/61), 1.05 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/chaRL/games/blackjack/chaRL/games/blackjack/chaRL/games/blackjack/chaRL/games/blackjack


In [ ]:
env = GameEnv()
model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("dqn_a2c_blackjack")

while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [9]:
from stable_baselines.common.env_checker import check_env
env = GameEnv()
check_env(env, warn=True)

/usr/local/lib/python3.6/dist-packages/stable_baselines/common/env_checker.py:47: UserWarning: The observation space is a Dict but the environment is not a gym.GoalEnv (cf https://github.com/openai/gym/blob/master/gym/core.py), this is currently not supported by Stable Baselines (cf https://github.com/hill-a/stable-baselines/issues/133), you will need to use a custom policy. 
  warnings.warn("The observation space is a Dict but the environment is not a gym.GoalEnv "


NameError: ignored